<a href="https://colab.research.google.com/github/RUBAIATH-E-ULFATH/Smishing-Detection-with-NLP-and-ML-Algorithms/blob/main/SMS_Phishing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Load Dataset & Import Necessary Modules

###Load Dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/spam.csv')

In [ ]:
df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


###Data preprocessing and cleaning


In [ ]:
y= df[["v1"]]

In [ ]:
y.head()

,v1
0,ham
1,ham
2,spam
3,ham
4,ham


In [ ]:
X=df[['v2']]

In [ ]:
X.head()

,v2
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
y.rename(columns={'v1': 'class'},inplace=True);




In [ ]:
X.rename(columns={'v2': 'text'},inplace=True);

In [ ]:
y.head()

,class
0,ham
1,ham
2,spam
3,ham
4,ham


In [ ]:
X.head()

,text
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
y.value_counts()

class
ham      4825
spam      747
dtype: int64

###Split Train - Test - Valdiation Set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify= y)

In [ ]:
X.shape


(5572, 1)

In [ ]:
X_train.shape


(4457, 1)

In [ ]:
y_train.shape

(4457, 1)

In [ ]:
X_test.shape

(1115, 1)

In [ ]:
y_test.shape

(1115, 1)

###Label Encode of the Categories

In [ ]:
y_train['class'] = y_train['class'].astype('category')
y_test['class'] = y_test['class'].astype('category')

In [ ]:
y_train['class'] = y_train['class'].str.replace('spam', '0')
y_train['class'] = y_train['class'].str.replace('ham', '1')

In [ ]:
y_test['class'] = y_test['class'].str.replace('spam', '0')
y_test['class'] = y_test['class'].str.replace('ham', '1')

###Converting to Lowecase and Removing Punctuations

In [ ]:

#X_train['text'] = X_train['text'].str.lower().str.replace('[^\w\s]', '')

In [ ]:
#X_test['text'] = X_test['text'].str.lower().str.replace('[^\w\s]', '')

###Feature Engineering (Stopwords, Lemmatization)

In [ ]:
from tqdm import tqdm_notebook as tqdm
from nltk import corpus
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer

In [ ]:
def Feature_Engineering(txt: str, stopwords: bool, lemmatization: bool):
    global total_row, progress_bar
    
    # Tokenizing
    tokenizer = WhitespaceTokenizer()
    txt = tokenizer.tokenize(txt.strip())
    
    # Stopwards
    if stopwords:
        stopwords_lst = corpus.stopwords.words("english")
        txt = (word for word in txt if word not in stopwords_lst)
        
    # Lemmatization
    if lemmatization:
        lemmatizer = WordNetLemmatizer()
        txt = (lemmatizer.lemmatize(word) for word in txt)
    
    # Making sentence
    txt = ' '.join(txt)
    
    
    return txt.strip()

In [ ]:

def X_Train_Feature_Engineering():
    global X_train
    X_train['text'] = X_train['text'].apply(
        lambda text: Feature_Engineering(
            txt=text,
            stopwords=True,
            lemmatization=False
        )
    )

In [ ]:
def X_Test_Feature_Engineering():
    global X_test
    X_test['text'] = X_test['text'].apply(
        lambda text: Feature_Engineering(
            txt = text,
            stopwords=True,
            lemmatization=False
        )
    )

In [ ]:
X_Train_Feature_Engineering()
X_Test_Feature_Engineering()

###Extra usefull columns creating

###Keeping copy of the processed dataset

In [ ]:
X_train_ORIG, y_train_ORIG = X_train.copy(), y_train.copy()
X_test_ORIG, y_test_ORIG = X_test.copy(), y_test.copy()

In [ ]:
df_train = pd.concat(
    [
        X_train_ORIG, 
        y_train_ORIG
    ],
    axis=1, 
    ignore_index=False
)

In [ ]:
df_test = pd.concat(
    [
        X_test_ORIG, 
        y_test_ORIG
    ],
    axis=1,
    ignore_index=False
)

In [ ]:
with pd.ExcelWriter('processed_data.xlsx') as writer: 
    df_train.to_excel(
        excel_writer=writer, 
        sheet_name='Train', 
        header=True, 
        index=False
    )
    df_test.to_excel(
        excel_writer=writer, 
        sheet_name='Test', 
        header=True, 
        index=False
    )

###Plotting Count of Targets

In [ ]:
# plt.figure(figsize=(6, 4))
# sns.set(
#     style='whitegrid',
#     color_codes=True,
# )
# sns.countplot(
#     x='text',
#     data=df_train,
#     hue='text'
# );

###Vectorizing

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(
    stop_words='english',
    max_features=10000,
    ngram_range=(1, 3)
)

In [ ]:
vectorizer.fit(X_train.text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=10000,
                min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
X_train = vectorizer.transform(X_train.text)

In [ ]:
# X_test = vectorizer.transform(X_test.text)

In [ ]:
X_train.shape

(4457, 10000)

In [ ]:
type(X_train)

scipy.sparse.csr.csr_matrix

In [ ]:
print(X_train)

  (0, 9388)	0.3068291413323403
  (0, 7534)	0.4314059499120256
  (0, 7502)	0.2267990711409743
  (0, 7261)	0.2670245247674841
  (0, 6526)	0.35137587972065953
  (0, 5155)	0.2724170343755342
  (0, 4033)	0.33551051990638836
  (0, 1022)	0.4314059499120256
  (0, 1020)	0.31725449856112653
  (1, 9550)	0.1597004929174256
  (1, 9306)	0.19394905479218547
  (1, 9305)	0.19394905479218547
  (1, 9302)	0.1597004929174256
  (1, 9049)	0.19394905479218547
  (1, 9048)	0.19394905479218547
  (1, 9028)	0.09055336961601938
  (1, 8168)	0.19394905479218547
  (1, 8167)	0.19394905479218547
  (1, 8166)	0.18722979667049638
  (1, 7121)	0.1741591075724096
  (1, 7064)	0.19394905479218547
  (1, 7063)	0.19394905479218547
  (1, 7062)	0.19394905479218547
  (1, 6118)	0.18201793516417383
  (1, 3712)	0.19394905479218547
  :	:
  (4452, 3875)	0.2912609973448165
  (4452, 3775)	0.34065208464257574
  (4452, 3761)	0.18572363405148087
  (4452, 3548)	0.3004151099487657
  (4452, 2371)	0.21775688824596004
  (4455, 8735)	0.5869744622523

###Keeping copy of vocabularies

In [ ]:
vocabulary = pd.DataFrame(
    tuple(vectorizer.vocabulary_.items()), 
    columns = ['word', 'id']
)

In [ ]:
vocabulary.to_excel("vocabulary_data.xlsx", header=True, index=False)

###Selecting Best Features

In [ ]:
from sklearn import feature_selection

In [ ]:
def Select_Best_Features(y: pd.core.series.Series, p_value_limit: int, feature_selector: str):
    global vectorizer

    if feature_selector.lower() == 'chi2':
        _, p = feature_selection.chi2(X_train, y)
    elif feature_selector.lower() == 'anova':
        _, p = feature_selection.f_classif(X_train, y)
    
    X_features = pd.DataFrame()
    X_features = X_features.append(
        pd.DataFrame(
            {
                'feature': vectorizer.get_feature_names(),
                'score': 1 - p,
            }
        )
    )
    
    return X_features[X_features['score'] > p_value_limit]

In [ ]:
X_features = Select_Best_Features(
    y=y_train['class'],
    p_value_limit=0.95,
    feature_selector='anova'
)

In [ ]:
X_features.to_excel("processed_vocabulary_data.xlsx", header=True, index=False)

###Final Selected Features

In [ ]:
vectorizer = TfidfVectorizer(
    vocabulary=X_features["feature"].unique().tolist()
)

In [ ]:
vectorizer.fit(X_train_ORIG.text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use...
                            '00 subs 16', '000', '000 bonus',
                            '000 bonus caller', '000 cash', '000 cash 000',
                            '000 cash await', '000 homeowners',
                            '000 homeowners tenants', '000 pounds',
                            '000 pounds txt', '000 prize', '000 prize claim',
                            '000 prize jackpot', '000 xmas', '000 xmas prize',
                            '01223585334', '01223585334 cum',
                            '01223585334

In [ ]:
X_train = vectorizer.transform(X_train_ORIG.text)

In [ ]:
X_test = vectorizer.transform(X_test_ORIG.text)

####Keeping copy of vocabularies

In [ ]:

vocabulary = pd.DataFrame(
    tuple(vectorizer.vocabulary_.items()), 
    columns = ['word', 'id']
)

In [ ]:
vocabulary.to_excel("vocabulary_data_2.xlsx", header=True, index=False)

In [ ]:
X_train.shape

(4457, 4319)

####Classifier

In [ ]:
from xgboost import XGBClassifier
from sklearn import metrics, naive_bayes, pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier



###XGBoost

In [ ]:

classifier = XGBClassifier(
    booster = 'gbtree',
    max_depth=5,
    n_estimators=5000,
    learning_rate=0.01,
    importance_type='gain',
    random_state= 0,
    n_jobs=8
)

In [ ]:
classifier.fit(X_train, y_train_ORIG)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=5000, n_jobs=8,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
prediction = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=y_test_ORIG, y_pred=prediction)
print(cm)

[[126  23]
 [  2 964]]


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy: {}%".format(round(accuracy_score(y_test_ORIG, prediction) * 100, 2)))

Accuracy: 97.76%


In [ ]:

print(
    classification_report(
        y_true=y_test_ORIG,
        y_pred=prediction,
        zero_division='warn',
        digits=3
    )
)

              precision    recall  f1-score   support

           0      0.984     0.846     0.910       149
           1      0.977     0.998     0.987       966

    accuracy                          0.978      1115
   macro avg      0.981     0.922     0.948      1115
weighted avg      0.978     0.978     0.977      1115



###Random forest

In [ ]:
classifier = RandomForestClassifier(
    n_estimators=1000, criterion='gini'
)

In [ ]:
classifier.fit(X_train, y_train_ORIG)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
prediction = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=y_test_ORIG, y_pred=prediction)
print(cm)

[[125  24]
 [  0 966]]


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy: {}%".format(round(accuracy_score(y_test_ORIG, prediction) * 100, 2)))

Accuracy: 97.85%


In [ ]:
print(
    classification_report(
        y_true=y_test_ORIG,
        y_pred=prediction,
        zero_division='warn',
        digits=3
    )
)

              precision    recall  f1-score   support

           0      1.000     0.839     0.912       149
           1      0.976     1.000     0.988       966

    accuracy                          0.978      1115
   macro avg      0.988     0.919     0.950      1115
weighted avg      0.979     0.978     0.978      1115



###Decision tree

In [ ]:
classifier= DecisionTreeClassifier(random_state=0, criterion='gini')

In [ ]:
classifier.fit(X_train, y_train_ORIG)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [ ]:
prediction = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=y_test_ORIG, y_pred=prediction)
print(cm)

[[127  22]
 [  9 957]]


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy: {}%".format(round(accuracy_score(y_test_ORIG, prediction) * 100, 2)))

Accuracy: 97.22%


In [ ]:
print(
    classification_report(
        y_true=y_test_ORIG,
        y_pred=prediction,
        zero_division='warn',
        digits=3
    )
)

              precision    recall  f1-score   support

           0      0.934     0.852     0.891       149
           1      0.978     0.991     0.984       966

    accuracy                          0.972      1115
   macro avg      0.956     0.922     0.938      1115
weighted avg      0.972     0.972     0.972      1115



###SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
classifier= SVC()

In [ ]:
classifier.fit(X_train, y_train_ORIG)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
prediction = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=y_test_ORIG, y_pred=prediction)
print(cm)

[[133  16]
 [  2 964]]


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy: {}%".format(round(accuracy_score(y_test_ORIG, prediction) * 100, 2)))

Accuracy: 98.39%


In [ ]:
print(
    classification_report(
        y_true=y_test_ORIG,
        y_pred=prediction,
        zero_division='warn',
        digits=3
    )
)

              precision    recall  f1-score   support

           0      0.985     0.893     0.937       149
           1      0.984     0.998     0.991       966

    accuracy                          0.984      1115
   macro avg      0.984     0.945     0.964      1115
weighted avg      0.984     0.984     0.984      1115

